In [2]:
install.packages(c("dplyr", "glmnet", "zoo", "lubridate"))


Installing packages into ‘/home/codespace/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)



In [4]:
library(dplyr)
library(glmnet)
library(zoo)
library(lubridate)

Loading required package: Matrix



Loaded glmnet 4.1-10


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [5]:
clean_oil_data = function(filename) {
  read.csv2(filename) %>% 
    filter(!is.na(Date)) %>% 
    # put Date into dateformat
    mutate(Date = as.Date(Date, "%d.%m.%Y")) %>% 
    # put all other variables into numer format
    mutate(across(-Date, ~ as.numeric(as.character(.))))
}

files = list(
  "Industry_D.csv", 
  "Industry_M.csv", 
  "Industry_W.csv", 
  "Macro_M.csv", 
  "StockPrices_M.csv"
)

cleaned_data = lapply(files, clean_oil_data)

industry_d = cleaned_data[[1]]
industry_m = cleaned_data[[2]] %>% 
# these variables already appear inside industry_d
  select(-any_of(c("CL1", "CL2", "Brent", "CRKS321C.Index", "Baltic.Dry.Index", "datadate")))
industry_w = cleaned_data[[3]]
macro_m = cleaned_data[[4]]
stockprices_m = cleaned_data[[5]]

head(industry_d)
head(industry_m)

,Date,CL1,CL2,Brent,CRKS321C.Index,BDIY.Index
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2024-07-30,74.73,73.88,78.63,21.883,1762
2,2024-07-29,75.81,74.80,79.78,22.171,1797
3,2024-07-26,77.16,76.00,81.13,21.071,1808
4,2024-07-25,78.28,77.11,82.37,20.832,1834
5,2024-07-24,77.59,76.58,81.71,19.776,1864
6,2024-07-23,76.96,75.92,81.01,21.118,1869


,Date,Daily.Production,Inventories,Rig.Count,Commercial.Long,Commercial.Short,Total.Open.Interest,X,X.1,X.2,X.3
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2024-06-28,13200,821134,581,1600085,1631509,3231594,NA,NA,NA,NA
2,2024-05-31,13100,826109,600,1750160,1785108,3535268,NA,NA,NA,NA
3,2024-04-30,13100,827161,613,1708427,1751709,3460136,NA,NA,NA,NA
4,2024-03-29,13100,815058,621,1687838,1725350,3413188,NA,NA,NA,NA
5,2024-02-29,13300,807417,626,1583286,1606750,3190036,NA,NA,NA,NA
6,2024-01-31,13000,779314,621,1673204,1695615,3368819,NA,NA,NA,NA


In [6]:
# set a grid for the date range
date_grid = data.frame(
  Date = seq(
    from = min(
      industry_d$Date,
      industry_w$Date,
      industry_m$Date,
      macro_m$Date,
      stockprices_m$Date,
      na.rm = TRUE
    ),
    to = max(
      industry_d$Date,
      industry_w$Date,
      industry_m$Date,
      macro_m$Date,
      stockprices_m$Date,
      na.rm = TRUE
    ),
    by = "day"
  )
)


In [7]:
# merge the data
data_merged = date_grid %>%
  left_join(industry_d, by = "Date") %>%
  left_join(industry_w, by = "Date") %>%
  left_join(industry_m, by = "Date") %>%
  left_join(macro_m, by = "Date") %>%
  left_join(stockprices_m, by = "Date") %>%
  arrange(Date)


head(data_merged)
tail(data_merged)
print(paste("Rows after merge:", nrow(data_merged)))

,Date,CL1,CL2,Brent,CRKS321C.Index,BDIY.Index,Weekly.Prod.Crude,Weekly.Rig.Count,Weekly.change.in.Crude.Stock,Weekly.Commercial.Long,⋯,DXY.returns,VIX,Gasoline.All.Grades,Dow.Jones.US.Oil.Gas.Index,ExxonMobil,ConocoPhilips,Chevron,BP,Shell,TotalEnergies
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1990-01-02,22.89,22.41,21.95,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1990-01-03,23.68,22.97,23.48,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,1990-01-04,23.41,22.53,26.78,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1990-01-05,23.08,22.03,27.40,NA,NA,7512,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,1990-01-06,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1990-01-07,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


,Date,CL1,CL2,Brent,CRKS321C.Index,BDIY.Index,Weekly.Prod.Crude,Weekly.Rig.Count,Weekly.change.in.Crude.Stock,Weekly.Commercial.Long,⋯,DXY.returns,VIX,Gasoline.All.Grades,Dow.Jones.US.Oil.Gas.Index,ExxonMobil,ConocoPhilips,Chevron,BP,Shell,TotalEnergies
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12624,2024-07-25,78.28,77.11,82.37,20.832,1834,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
12625,2024-07-26,77.16,76.00,81.13,21.071,1808,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
12626,2024-07-27,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
12627,2024-07-28,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
12628,2024-07-29,75.81,74.80,79.78,22.171,1797,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
12629,2024-07-30,74.73,73.88,78.63,21.883,1762,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] "Rows after merge: 12629"


In [19]:
data_merged_filled = data_merged %>%
  arrange(Date) %>%
  mutate(across(-Date, ~ zoo::na.locf(., na.rm = FALSE))) %>% 
  mutate(across(-Date, ~ zoo::na.locf(., fromLast = TRUE, na.rm = FALSE)))

head(data_merged_filled)  # earliest dates
tail(data_merged_filled)  # latest dates
print(paste("Rows after filling:", nrow(data_merged_filled)))

print(paste("Variables in dataset:", paste(names(data_merged_filled), collapse = ", ")))


,Date,CL1,CL2,Brent,CRKS321C.Index,BDIY.Index,Weekly.Prod.Crude,Weekly.Rig.Count,Weekly.change.in.Crude.Stock,Weekly.Commercial.Long,⋯,DXY.returns,VIX,Gasoline.All.Grades,Dow.Jones.US.Oil.Gas.Index,ExxonMobil,ConocoPhilips,Chevron,BP,Shell,TotalEnergies
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1990-01-02,22.89,22.41,21.95,1.777,1599,7512,532,9539,166945,⋯,-0.004610635,13.2,1.125,107.21,15.1875,12.483,22.3125,19.9688,61.28,14.5116
2,1990-01-03,23.68,22.97,23.48,1.777,1599,7512,532,9539,166945,⋯,-0.004610635,13.2,1.125,107.21,15.1875,12.483,22.3125,19.9688,61.28,14.5116
3,1990-01-04,23.41,22.53,26.78,1.777,1599,7512,532,9539,166945,⋯,-0.004610635,13.2,1.125,107.21,15.1875,12.483,22.3125,19.9688,61.28,14.5116
4,1990-01-05,23.08,22.03,27.40,1.777,1599,7512,532,9539,166945,⋯,-0.004610635,13.2,1.125,107.21,15.1875,12.483,22.3125,19.9688,61.28,14.5116
5,1990-01-06,23.08,22.03,27.40,1.777,1599,7512,532,9539,166945,⋯,-0.004610635,13.2,1.125,107.21,15.1875,12.483,22.3125,19.9688,61.28,14.5116
6,1990-01-07,23.08,22.03,27.40,1.777,1599,7512,532,9539,166945,⋯,-0.004610635,13.2,1.125,107.21,15.1875,12.483,22.3125,19.9688,61.28,14.5116


,Date,CL1,CL2,Brent,CRKS321C.Index,BDIY.Index,Weekly.Prod.Crude,Weekly.Rig.Count,Weekly.change.in.Crude.Stock,Weekly.Commercial.Long,⋯,DXY.returns,VIX,Gasoline.All.Grades,Dow.Jones.US.Oil.Gas.Index,ExxonMobil,ConocoPhilips,Chevron,BP,Shell,TotalEnergies
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12624,2024-07-25,78.28,77.11,82.37,20.832,1834,13300,482,-3741,654310,⋯,0.01135205,12.44,3.557,760.4,115.12,114.38,156.42,36.1,72.18,66.68
12625,2024-07-26,77.16,76.00,81.13,21.071,1808,13300,482,-3741,654310,⋯,0.01135205,12.44,3.557,760.4,115.12,114.38,156.42,36.1,72.18,66.68
12626,2024-07-27,77.16,76.00,81.13,21.071,1808,13300,482,-3741,654310,⋯,0.01135205,12.44,3.557,760.4,115.12,114.38,156.42,36.1,72.18,66.68
12627,2024-07-28,77.16,76.00,81.13,21.071,1808,13300,482,-3741,654310,⋯,0.01135205,12.44,3.557,760.4,115.12,114.38,156.42,36.1,72.18,66.68
12628,2024-07-29,75.81,74.80,79.78,22.171,1797,13300,482,-3741,654310,⋯,0.01135205,12.44,3.557,760.4,115.12,114.38,156.42,36.1,72.18,66.68
12629,2024-07-30,74.73,73.88,78.63,21.883,1762,13300,482,-3741,654310,⋯,0.01135205,12.44,3.557,760.4,115.12,114.38,156.42,36.1,72.18,66.68


[1] "Rows after filling: 12629"
[1] "Variables in dataset: Date, CL1, CL2, Brent, CRKS321C.Index, BDIY.Index, Weekly.Prod.Crude, Weekly.Rig.Count, Weekly.change.in.Crude.Stock, Weekly.Commercial.Long, Weekly.Commercial.Short, Weekly.Total.Open.Interest, Daily.Production, Inventories, Rig.Count, Commercial.Long, Commercial.Short, Total.Open.Interest, X, X.1, X.2, X.3, CPI.YOY.., X3M.Yield, X10Y.Yield, IndustrialProduction.Index, SPX, DXY.returns, VIX, Gasoline.All.Grades, Dow.Jones.US.Oil.Gas.Index, ExxonMobil, ConocoPhilips, Chevron, BP, Shell, TotalEnergies"


In [20]:
# --- define variable groups ---
price_vars = c(
  "CL1", "CL2", "Brent",
  "CRKS321C.Index", "Baltic.Dry.Index",
  "SPX",
  "Gasoline.All.Grades",
  "Dow.Jones.US.Oil.Gas.Index",
  "ExxonMobil", "ConocoPhilips", "Chevron",
  "BP", "Shell", "TotalEnergies"
)

macro_diff_vars = c(
  "CPI.YOY..", "X3M.Yield", "X10Y.Yield",
  "Industrial.Production.Index",
  "DXY.returns", "VIX"
)

quantity_diff_vars = c(
  "Daily.Production", "Inventories", "Rig.Count",
  "Commercial.Long", "Commercial.Short", "Total.Open.Interest"
)

# --- create transformed dataset ---
data_merged_final = data_merged_filled %>%
  arrange(Date) %>%
  mutate(
    # log returns for prices
    across(all_of(price_vars),
           ~ log(.) - log(lag(.)),
           .names = "{.col}_log_ret"),
    
    # differences for macro variables
    across(all_of(macro_diff_vars),
           ~ . - lag(.),
           .names = "{.col}_diff"),
    
    # differences for quantities / positions
    across(all_of(quantity_diff_vars),
           ~ . - lag(.),
           .names = "{.col}_diff")
  ) %>%
  # keep only observations where the dependent variable is defined
  filter(!is.na(CL1_log_ret))

head(data_merged_final)
tail(data_merged_final)

ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `across(all_of(price_vars), ~log(.) - log(lag(.)), .names
  = "{.col}_log_ret")`.
[1mCaused by error in `across()`:[22m
[1m[22m[36mℹ[39m In argument: `all_of(price_vars)`.
[1mCaused by error in `all_of()`:[22m
[33m![39m Can't subset elements that don't exist.
[31m✖[39m Element `Baltic.Dry.Index` doesn't exist.


In [16]:
# split the data. first economic cycle of 10 years completes EOY 2000
IS_data = data_merged_final %>%
    filter(Date <= as.Date("2000-12-31"))
OoS_data = data_merged_final %>%
    filter(Date > as.Date("2000-12-31"))

In [17]:
# Identify initial predictor columns
predictors_initial = setdiff(names(IS_data), c("Date", "CL1_log_ret", "CL2_log_ret", "CL1", "CL2"))

# Calculate SD and find "Dead" columns (SD = 0)
is_sds_check = apply(IS_data[, predictors_initial], 2, sd, na.rm = TRUE)
dead_cols = names(is_sds_check)[is_sds_check == 0 | is.na(is_sds_check)]
predictors = setdiff(predictors_initial, dead_cols)

if(length(dead_cols) > 0) {
  print("Removing columns with zero variance:")
  print(dead_cols)
}

# Calculate mean and SD from IS only (for active predictors)
is_means = colMeans(IS_data[, predictors])
is_sds   = apply(IS_data[, predictors], 2, sd)

# Create the final tables by selecting only Date, Targets, and Active Predictors
final_cols = c("Date", "CL1_log_ret", "CL2_log_ret", "CL1", "CL2", predictors)
IS_data_scaled = IS_data[, final_cols]
OoS_data_scaled = OoS_data[, final_cols]

# Apply the scaling to the predictors only
IS_data_scaled[, predictors] = as.data.frame(scale(IS_data[, predictors], center = is_means, scale = is_sds))
OoS_data_scaled[, predictors] = as.data.frame(scale(OoS_data[, predictors], center = is_means, scale = is_sds))

# Final Verification
print(paste("Columns in original:", ncol(IS_data)))
print(paste("Columns in final:", ncol(IS_data_scaled)))
print(paste("Is Shell still there?:", "Shell" %in% names(IS_data_scaled)))

head(IS_data_scaled)
tail(OoS_data_scaled)

[1] "Removing columns with zero variance:"
 [1] "X"             "X.1"           "X.2"           "X.3"          
 [5] "Shell"         "X_log_ret"     "X.1_log_ret"   "X.2_log_ret"  
 [9] "X.3_log_ret"   "Shell_log_ret"


[1] "Columns in original: 73"
[1] "Columns in final: 63"
[1] "Is Shell still there?: FALSE"


,Date,CL1_log_ret,CL2_log_ret,CL1,CL2,Brent,CRKS321C.Index,BDIY.Index,Weekly.Prod.Crude,Weekly.Rig.Count,⋯,SPX_log_ret,DXY.returns_log_ret,VIX_log_ret,Gasoline.All.Grades_log_ret,Dow.Jones.US.Oil.Gas.Index_log_ret,ExxonMobil_log_ret,ConocoPhilips_log_ret,Chevron_log_ret,BP_log_ret,TotalEnergies_log_ret
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1990-01-03,0.03393067,0.02468173,23.68,22.97,0.7339079,-0.7620487,0.6785869,1.84972,1.671564,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
2,1990-01-04,-0.01146753,-0.01934126,23.41,22.53,1.3756573,-0.7620487,0.6785869,1.84972,1.671564,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
3,1990-01-05,-0.01419684,-0.02244259,23.08,22.03,1.4962284,-0.7620487,0.6785869,1.84972,1.671564,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
4,1990-01-06,0.00000000,0.00000000,23.08,22.03,1.4962284,-0.7620487,0.6785869,1.84972,1.671564,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
5,1990-01-07,0.00000000,0.00000000,23.08,22.03,1.4962284,-0.7620487,0.6785869,1.84972,1.671564,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
6,1990-01-08,-0.06534763,-0.04645517,21.62,21.03,1.0256122,-0.7620487,0.6785869,1.84972,1.671564,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294


,Date,CL1_log_ret,CL2_log_ret,CL1,CL2,Brent,CRKS321C.Index,BDIY.Index,Weekly.Prod.Crude,Weekly.Rig.Count,⋯,SPX_log_ret,DXY.returns_log_ret,VIX_log_ret,Gasoline.All.Grades_log_ret,Dow.Jones.US.Oil.Gas.Index_log_ret,ExxonMobil_log_ret,ConocoPhilips_log_ret,Chevron_log_ret,BP_log_ret,TotalEnergies_log_ret
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8605,2024-07-25,0.00885359,0.006897028,78.28,77.11,12.18622,12.63635,1.430187,13.88465,1.245756,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
8606,2024-07-26,-0.01441095,-0.014499633,77.16,76.00,11.94508,12.80440,1.347031,13.88465,1.245756,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
8607,2024-07-27,0.00000000,0.000000000,77.16,76.00,11.94508,12.80440,1.347031,13.88465,1.245756,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
8608,2024-07-28,0.00000000,0.000000000,77.16,76.00,11.94508,12.80440,1.347031,13.88465,1.245756,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
8609,2024-07-29,-0.01765098,-0.015915455,75.81,74.80,11.68254,13.57786,1.311850,13.88465,1.245756,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294
8610,2024-07-30,-0.01434859,-0.012375729,74.73,73.88,11.45890,13.37535,1.199909,13.88465,1.245756,⋯,-0.04344996,NaN,-0.008152005,-0.01196843,-0.03267233,-0.04142982,-0.01421578,-0.02000749,-0.02583869,-0.02227294


In [18]:
# 1. Generate Lags (1 to 90) for all standardized predictors
max_lag = 90
lag_list = list()

for (i in 1:max_lag) {
  # Shift the standardized predictors from your scaled dataframes
  is_orig = IS_data_scaled[, predictors]
  is_lagged = as.data.frame(lapply(is_orig, function(x) c(rep(NA, i), head(x, -i))))
  colnames(is_lagged) = paste0(predictors, "_L", i)
  
  oos_orig = OoS_data_scaled[, predictors]
  oos_lagged = as.data.frame(lapply(oos_orig, function(x) c(rep(NA, i), head(x, -i))))
  colnames(oos_lagged) = paste0(predictors, "_L", i)
  
  lag_list$IS[[i]] = is_lagged
  lag_list$OoS[[i]] = oos_lagged
}

# 2. Combine lags into huge dataframes using cbind
IS_huge = cbind(IS_data_scaled, do.call(cbind, lag_list$IS))
OoS_huge = cbind(OoS_data_scaled, do.call(cbind, lag_list$OoS))

# 3. Clean up NAs using indexing (Removes the first 90 rows)
IS_huge = IS_huge[(max_lag + 1):nrow(IS_huge), ]
OoS_huge = OoS_huge[(max_lag + 1):nrow(OoS_huge), ]

# 4. Prepare the Matrices for LASSO
all_names = names(IS_huge)
lagged_predictor_names = all_names[grepl("_L", all_names)]

x_train = as.matrix(IS_huge[, lagged_predictor_names])
# Multiply y by 100 to convert to percentage returns for numerical stability
y_train_scaled = IS_huge$CL1_log_ret * 100

x_test = as.matrix(OoS_huge[, lagged_predictor_names])
y_test = OoS_huge$CL1_log_ret

# 5. Run Cross-Validation LASSO
set.seed(123)
# nlambda and lambda.min.ratio are adjusted to fix the 'dotted line on the edge' issue
cv_lasso = cv.glmnet(x_train, y_train_scaled, alpha = 1, nlambda = 200, lambda.min.ratio = 0.0001)

# Plot the error curve
plot(cv_lasso)

# 6. Extract Coefficients at lambda.min
best_coefs = coef(cv_lasso, s = "lambda.min")

coef_df = data.frame(
  Variable = rownames(best_coefs),
  Coefficient = as.numeric(best_coefs)
)

# Filter and sort
coef_df = coef_df[coef_df$Coefficient != 0 & coef_df$Variable != "(Intercept)", ]
# Adjust coefficients back to decimal scale since we multiplied Y by 100
coef_df$Coefficient = coef_df$Coefficient / 100
coef_df = coef_df[order(-abs(coef_df$Coefficient)), ]

print("Lags selected by LASSO:")
print(coef_df)

ERROR: Error in glmnet(x, y, weights = weights, offset = offset, lambda = lambda, : x has missing values; consider using makeX() to impute them
